In [70]:
from langgraph.graph  import StateGraph ,START, END
import os
import google.generativeai as genai # Importing the necessary modules
from typing import TypedDict, Dict ,Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator

In [71]:
load_dotenv()  # Load environment variables from .env file

True

In [72]:
google_api_key = os.getenv("google_api_key")  # Fetch the API key from environment variables
genai.configure(api_key=google_api_key)  # Configure the Google Generative AI client
model = genai.GenerativeModel("gemini-2.5-flash")  # Specify the model to use

In [73]:
class evaluationSchema(BaseModel):
    """
    Schema for essay evaluation.
    """

    score: int=Field (description="Score for the essay, ranging from 0 to 10")
    feedback: str= Field(description="Feedback on the essay")

In [74]:
class upseState(TypedDict):
    """
    TypedDict for UPSE state.
    """
    essay: str
    language_Feedback: str
    clarity_Feedback: str
    analysis_Feedback: str
    overall_Feedback: str
    individual_Score: Annotated[list[int],operator.add]

In [75]:


     # Corrected code
def clarity_of_thought(state: dict) -> dict:
    # `state` is now a dictionary containing the current graph state
    # You can access values from it, for example:
    essay = state.get('essay')

    # ... your logic to get a `response` object ...
    response = model.generate_content(
        "Provide clarity feedback for the following essay:\n{essay}"

    )

    # Return a dictionary with the fields you want to update.
    # LangGraph will merge this into the main state.
    return {"response_content": response.text.strip()}

    

    

In [76]:
def analysis_feedback(essay: str) -> str:
    
    #Function to provide analysis feedback on the essay.
    
    response = model.generate_content(
        "Provide analysis feedback for the following essay:\n{essay}")
        
    return {'analysis': response.text.strip()}
    print(response)

In [77]:
def language_feedback(essay: str) -> str:
    """
    Function to provide language feedback on the essay.
    """
    response = model.generate_content(
        "Provide language feedback for the following essay:\n{essay}")
        
    return {'language': response.text.strip()}

In [78]:
def overall_feedback(essay:str)-> str:
    """ This function takes an essay as input and returns a string with the overall feedback on the essay by llm."""
    # Initialize the feedback string
    response = model.generate_content(
        "Provide a feedback on the following essay and rate it out of 10 :\n {essay}"
    )
    return {'overall': response.text.strip()}
    

In [79]:
# The rest of the code would typically define the state graph and its transitions,
# but since the original code does not include that, we will not add it here.
# defining nodes and edges for the state graph
graph = StateGraph(upseState)
graph.add_node('clarity_of_thought',clarity_feedback)
graph.add_node('language', language_feedback)
graph.add_node('analysis', analysis_feedback)
graph.add_node('overall', overall_feedback)

# Define edge(connection) between states
graph.add_edge(START, 'clarity_of_thought')
graph.add_edge(START, 'language')
graph.add_edge(START, 'analysis')
graph.add_edge( 'analysis','overall')  
graph.add_edge('clarity_of_thought', 'overall')
graph.add_edge('language', 'overall')
graph.add_edge('overall', END)
workflow=graph.compile()



In [80]:

initial_state = {
    'essay': """## The Dual Helix: India's Political Structure and Foreign Policy in a Dynamic World

As a candidate preparing for the Union Public Service Commission examination, understanding the intricate interplay between India's domestic political architecture and its external engagements is paramount. India, the world's largest democracy, navigates a complex global landscape, with its foreign policy being a direct reflection of its constitutional principles, domestic imperatives, and historical legacy. This essay will dissect the salient features of India's political structure and its evolving foreign policy, highlighting their symbiotic relationship and the challenges they face in the 21st century.

India's political structure is a testament to the foresight of its founding fathers, who crafted a unique system designed to manage unparalleled diversity and uphold democratic values. At its core, India is a **Sovereign, Socialist, Secular, Democratic Republic**, as enshrined in its Constitution. It operates as a **parliamentary democracy** modelled on the Westminster system, where the Executive is accountable to the Legislature. The Prime Minister, as the real head of government, and the Council of Ministers, derive their authority from the Lok Sabha, the directly elected lower house of Parliament. This parliamentary system, characterized by collective responsibility and frequent elections, ensures a degree of responsiveness to public sentiment, crucial for a nation of over 1.4 billion people.

Concomitantly, India functions as a **quasi-federal state**, described as a "Union of States." While exhibiting a strong unitary bias, particularly during emergencies or through the distribution of legislative powers, it grants significant autonomy to its states. The division of powers between the Union and State Lists, the existence of independent state legislatures, and the provision for state-level elections are hallmarks of its federal character. This **cooperative federalism** allows for regional variations in governance and policy implementation, addressing the diverse needs of India's linguistic, cultural, and geographical landscape. However, it also presents challenges in Centre-state relations, particularly concerning fiscal transfers, resource sharing, and cross-border issues, which can indirectly influence foreign policy initiatives requiring state cooperation (e.g., border trade, land acquisition for projects).

The **judiciary**, headed by the Supreme Court, stands as an independent pillar, acting as the guardian of the Constitution and the protector of fundamental rights. Its power of judicial review ensures the executive and legislative actions adhere to constitutional provisions, providing a crucial check and balance. Furthermore, institutions like the Election Commission, Comptroller and Auditor General (CAG), and Union Public Service Commission (UPSC) reinforce the democratic framework, ensuring fair electoral processes, financial accountability, and merit-based public service. This robust institutional setup, despite its imperfections and the occasional strains of coalition politics or populist pressures, provides the essential domestic stability and legitimacy for India to project its influence on the global stage.

India's foreign policy has evolved significantly since independence, transitioning from a principled stance of **Non-Alignment** to a more pragmatic and multi-aligned approach often termed **Strategic Autonomy**. In the Cold War era, Non-Alignment, championed by Jawaharlal Nehru, was a moral imperative born out of newly independent nations' desire to avoid entanglement in superpower blocs. It allowed India to pursue an independent foreign policy, focusing on nation-building and advocating for decolonization, disarmament, and a New International Economic Order. Panchsheel, the five principles of peaceful coexistence, formed the bedrock of its initial engagements.

With the end of the Cold War and the advent of a multipolar world, India's foreign policy underwent a recalibration. While the spirit of non-alignment – maintaining decision-making independence – persists, its application has transformed. Today, India pursues a policy of **multi-alignment**, engaging with diverse global powers and blocs based on convergences of national interest rather than ideological alignment. This pragmatic approach is driven by several key imperatives:

1.  **Economic Development:** Securing foreign investment, diversifying trade partnerships, ensuring energy security, and integrating into global supply chains are paramount. This drives robust engagements with economic powerhouses like the United States, European Union, Japan, and the ASEAN bloc. Initiatives like the "Act East" policy emphasize economic and strategic ties with Southeast and East Asia.
2.  **National Security:** Addressing traditional and non-traditional threats remains a core concern. This includes counter-terrorism cooperation, securing maritime trade routes, and managing complex border disputes, particularly with Pakistan and China. Strategic partnerships with countries like the US, Russia, France, and Israel in defense cooperation and intelligence sharing are crucial. The growing focus on the **Indo-Pacific** region, reflected in forums like the Quad (India, US, Japan, Australia), underscores a commitment to a free, open, and rules-based maritime order.
3.  **Regional Stability and Neighbourhood First:** India's foreign policy prioritizes its immediate neighbourhood. The "Neighbourhood First" policy seeks to foster connectivity, economic cooperation, and people-to-people ties with South Asian countries, despite persistent challenges with some neighbours. Participation in regional groupings like BIMSTEC (Bay of Bengal Initiative for Multi-Sectoral Technical and Economic Cooperation) aims to enhance regional prosperity and stability.
4.  **Global Governance and Multilateralism:** India remains a steadfast proponent of a rules-based international order and reformed multilateralism. It actively participates in the United Nations, G20, BRICS, and other global forums, advocating for a more equitable global order, climate justice, sustainable development, and reforms of international institutions to reflect contemporary realities. India's aspirations for a permanent seat in the UNSC are a testament to its desire for a greater voice in global decision-making.

The interplay between India's political structure and foreign policy is profound. The stability provided by a parliamentary majority at the Centre allows for decisive foreign policy actions and long-term strategic planning. India's democratic credentials lend it significant **soft power** and legitimacy on the global stage, making it a preferred partner for many nations. The vibrant media, active civil society, and robust parliamentary debates ensure a degree of public scrutiny over foreign policy decisions, contributing to their legitimacy and sustainability. Conversely, foreign policy successes, such as securing strategic partnerships or attracting investments, reinforce domestic stability and contribute to the government's credibility.

However, challenges persist. Domestically, managing regional aspirations and coalition dynamics can at times impact foreign policy coherence. The quasi-federal structure means that states, especially those bordering other countries, have a direct stake in foreign policy issues like cross-border trade, migration, and security, requiring delicate coordination. Externally, navigating the complexities of great power competition, managing volatile neighbourhood dynamics, ensuring energy and resource security, and addressing trans-national issues like climate change and pandemics require nimble diplomacy and significant internal consensus.

In conclusion, India's political structure, characterized by a robust parliamentary democracy and a dynamic quasi-federal system, provides the foundational stability and legitimacy necessary for a vibrant foreign policy. From the idealism of non-alignment to the pragmatism of strategic autonomy, India's external engagements are increasingly shaped by its developmental imperatives, security concerns, and aspiration to be a leading power in a multipolar world. As India progresses, its ability to leverage its democratic strengths, maintain internal cohesion, and skillfully navigate the complexities of international relations will be critical in securing its national interests and contributing meaningfully to global peace and prosperity. This dual helix of domestic governance and external engagement will continue to define India's trajectory in the 21st century."""
}

final_state=workflow.invoke(initial_state)
print(final_state)

# Assuming 'workflow' is defined elsewhere in your code
# final_state = workflow.invoke(initial_state) 
# print(final_state)


{'essay': '## The Dual Helix: India\'s Political Structure and Foreign Policy in a Dynamic World\n\nAs a candidate preparing for the Union Public Service Commission examination, understanding the intricate interplay between India\'s domestic political architecture and its external engagements is paramount. India, the world\'s largest democracy, navigates a complex global landscape, with its foreign policy being a direct reflection of its constitutional principles, domestic imperatives, and historical legacy. This essay will dissect the salient features of India\'s political structure and its evolving foreign policy, highlighting their symbiotic relationship and the challenges they face in the 21st century.\n\nIndia\'s political structure is a testament to the foresight of its founding fathers, who crafted a unique system designed to manage unparalleled diversity and uphold democratic values. At its core, India is a **Sovereign, Socialist, Secular, Democratic Republic**, as enshrined in